## Bank client behavior prediction

### About dataset

- For this analysis there are two files in the `data` folder `credit_record.csv` and `application_record.csv` where bank clients are related by the `ID` column.

- In `application_record.csv` we have the following variables

| Feature Name         | Explanation     | Additional Remarks |
|--------------|-----------|-----------|
| ID | Randomly allocated client number      |         |
| AMT_INCOME   | Annual income  |  |
| NAME_INCOME_TYPE   | Income Source |  |
| NAME_EDUCATION_TYPE   | Level of Education  |  |
| CODE_GENDER   | Applicant's Gender   |  |
| FLAG_OWN_CAR | Car Ownership |  | 
| CNT_CHILDREN | Number of Children | |
| FLAG_OWN_REALTY | Real Estate Ownership | | 
| NAME_FAMILY_STATUS | Relationship Status | | 
| NAME_HOUSING_TYPE | Housing Type | | 
| DAYS_BIRTH | No. of Days | Count backwards from current day (0), -1 means yesterday
| DAYS_EMPLOYED | No. of Days | Count backwards from current day (0). If positive, it means the person is currently unemployed.
| FLAG_MOBIL | Mobile Phone Ownership | | 
| FLAG_WORK_PHONE | Work Phone Ownership | | 
| FLAG_PHONE | Landline Phone Ownership | | 
| FLAG_EMAIL | Landline Phone Ownership | | 
| OCCUPATION_TYPE | Occupation | | 
| CNT_FAM_MEMBERS | Count of Family Members | |



- In `credit_record.csv` we have the following variables


| Feature Name         | Explanation     | Additional Remarks |
|--------------|-----------|-----------|
| ID | Randomly allocated client number | |
| MONTHS_BALANCE | Number of months in the past from now when STATUS is measured | 0 = current month, -1 = last month, -2 = two months ago, etc.|
| STATUS | Number of days a payment is past due | 0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month |

---
---

### Part 1. Reading, Summarising and Cleaning Data

This part involved importing the datasets into Dataframes and determining the number of rows and unique bank clients in each. I then merged the two datasets on a common ID column, applying an inner join to focus only on matching records, creating a consolidated DataFrame.

This allowed me to explore the merged data, analyze the number of rows and unique clients, and investigate differences in multiple rows associated with the same ID.

In [40]:
import pandas as pd

df_application = pd.read_csv('data/application_record.csv')
df_credit = pd.read_csv('data/credit_record.csv')

#2. Printing number of rows os dataframes:
print(f'There are {df_application.shape[0]} rows in df_application')
print(f'There are {df_credit.shape[0]} rows in df_credit')

#3. Printing number of unique clients:
print(f'There are {df_application['ID'].nunique()} unique clients in df_application')
print(f'There are {df_credit['ID'].nunique()} unique clients in df_credit')

#4. Merging two dataframes into one, using ID columns:
df = pd.merge(df_application, df_credit, on='ID', how='inner')

#5. Checking number of rows and unique clients in new dataframe:
#df.shape
#df['ID'].nunique()

#6. Checking for the duplicated rows difference: 
#duplicates = df.duplicated(subset=['ID'])
#print(df[duplicates])

There are 438445 rows in df_application
There are 1047185 rows in df_credit
There are 438398 unique clients in df_application
There are 45924 unique clients in df_credit


#### Part 1 outcome:
1. Number of rows in dataframes:
    - There are **438,445** rows in `df_application` dataframe.
    - There are has **1,047,185** rows in `df_credit` dataframe.
2. Number of unique clients in both dataframes:
    - There are **438,398** unique clients in `df_application` dataframe.
    - There are **45,924** unique clients in `df_credit` dataframe.
3. `df` dataframe details after merge:
    - There are **776,325** rows in `df` dataframe after the merge.
    - There are **36,396** unique clients in `df` dataframe after the merge.
4. Explanation on multiple rows for each ID in `df` dataframe:
    - One user can have multiple record of payment status and monthly balance over time.
    - Over time informations like employment status, occupation, income, family status can change.
    - Therefore, multiple IDs are different as they represent different records of one client. 

---
---

### Part 2. Data Wrangling and Feature Engineering for Client Overdue Analysis
After merging these datasets based on a common ID column, I performed a series of data wrangling tasks to identify clients with overdue payments in the last 12 months. This involved mapping the STATUS column to indicate whether a client had past-due payments and filtering the data accordingly. I utilized NumPy to extract unique client IDs with overdue statuses and created a new Dataframe containing these clients, ensuring duplicates were removed and missing values handled efficiently.

I then appended additional rows to balance the dataset to a required size, while marking each client with an appropriate target variable. 

In [41]:
import numpy as np

#1. Changing STATUS with manual mapping:
mapping = {'C': 0, 'X': 0, '0': 0,'1': 1, '2': 1, '3': 1, '4': 1, '5': 1}
df['STATUS'] = df['STATUS'].map(mapping)

#2. Filtering the df where STATUS is 1 during last 12 months:
last_year_status = df.loc[(df['STATUS'] == 1) & (df['MONTHS_BALANCE'] >= -11)]

#Converting the unique IDs to a numpy array
list_of_past_due = np.array(last_year_status['ID'].unique())

#3. Creating new dataframe with IDs in list_of_past_due:
df_final = df.loc[df['ID'].isin(list_of_past_due)]

# Deleting duplicated rows, only keeping first occurance: 
df_final = df_final.drop_duplicates(subset=['ID'], keep='first')

print(f'There are {df_final.shape[0]} rows in df_final')

#4. Adding new column for all rows in df_final:
df_final['y'] = 1

#5. Filtering rows with unique ID that are not in list_of_past_due:
df_add = df[~df['ID'].isin(list_of_past_due)].drop_duplicates(subset=['ID'], keep='first')

# Number of rows needed:
rows = 4500 - df_final.shape[0]

# Selecting only rows that required:
df_required = df_add.iloc[:rows]

# Adding rows to df_final:
df_final = pd.concat([df_final, df_required], ignore_index=True)

#6. Filling missing values with zeros:
df_final['y'] = df_final['y'].fillna(0)

# Removing the 'STATUS' and 'MONTHS_BALANCE' columns from df_final
df_final.drop(columns=['STATUS', 'MONTHS_BALANCE'], inplace=True)

There are 1737 rows in df_final


---
---

### Part 3. Data Cleaning and Variable Classification for Final Dataset Preparation
In this part, I focused on refining the dataset df_final by first removing the ID column, as it was no longer necessary for further analysis, and then resetting the index to ensure a clean and continuous structure. 

Next, I used pandas functions to check for missing values within the dataset. This process involved calculating the number of missing values in each column and sorting the results to assess which features required further attention. 

In [42]:
#1. Deleting ID column from df_final:
df_final.drop(columns=['ID'], inplace=True)

# Resetting index of df_final:
df_final.reset_index(drop=True, inplace=True)

#3. Finding missing values: 
#df_final.isnull().sum().sort_values()

#### Part 3 markdown:
1. There are 5 numeric and 12 nominal and 1 ordinal variables in `df_final`. Names of variables of each types are shown in following table:
   
|Variable type|Numbers of features|Features' list|
| --- | --- | --- |
|Numeric:|5|CNT_CHILDREN, AMT_INCOME, DAYS_BIRTH, DAYS_EMPLOYED, CNT_FAM_MEMBERS|
|Ordinal:|1| NAME_EDUCATION_TYPE |
|Nominal:|12|CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_INCOME_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, FLAG_MOBIL, FLAG_WORK_PHONE, FLAG_PHONE, FLAG_EMAIL, OCCUPATION_TYPE, y|

2. Using `isnull().sum().sort_values()` function I sorted which particular variables have missing values. This method is more suitable because it helps us exclude features which does not have missing values and identify and focus on features with missing values. Total number of missing values of each variables are shown in following table: 
   
|Feature name|Variable type|Number of missing values|
| --- | --- | --- |
|CNT_CHILDREN|Numeric|74|
|OCCUPATION_TYPE|Nominal|1354|
|NAME_EDUCATION_TYPE|Ordinal|1831|

From the table we can see that we have 3 features with missing values: 
- CNT_CHILDREN is numeric variable as it is number if children we can't use mean, therefore will impute missing values with median.
- OCCUPATION_TYPE is nominal feature, thus will impute missing values with mode.
- NAME_EDUCATION_TYPE is ordinal feature, thus will mpute missing values with mode.1


---
---

### Part 4. Imputing missing values and dealing with categorical features



In this part handled missing values in a dataset by applying different imputation techniques based on the nature of the variables. For numerical data, I used the median to fill in missing values, ensuring robustness against outliers. For nominal data, I applied the mode to maintain the most common category. Similarly, for ordinal data in NAME_EDUCATION_TYPE, I used the mode to preserve the order and frequency of the values. This careful imputation process ensured that the dataset was complete and ready for accurate analysis, while maintaining the integrity of each variable type.

In [43]:
# Imputing missing values in CNT_CHILDREN with the mean, since it is numerical data:
df_final['CNT_CHILDREN'].fillna(df_final['CNT_CHILDREN'].median(), inplace=True)

# Imputing missing values in OCCUPATION_TYPE with the mode, since it is nominal data:
df_final['OCCUPATION_TYPE'].fillna(df_final['OCCUPATION_TYPE'].mode()[0], inplace=True)

# Imputing missing values in NAME_EDUCATION_TYPE with the mode, since it is ordinal data:
df_final['NAME_EDUCATION_TYPE'].fillna(df_final['NAME_EDUCATION_TYPE'].mode()[0], inplace=True)

---
---

### Part 5. Ordinal Variable Conversion for Educational Levels

Converting the values in `NAME_EDUCATION_TYPE` as follows
- Lower secondary -> 1
- Secondary / secondary special -> 2
- Incomplete higher -> 3
- Higher education -> 4

In [44]:
# Using mapping to convert, since it is ordinal data
edu_mapping = {'Lower secondary': 1, 'Secondary / secondary special': 2, 'Incomplete higher': 3, 'Higher education': 4}
df_final['NAME_EDUCATION_TYPE'] = df_final['NAME_EDUCATION_TYPE'].replace(edu_mapping)

---
---

### Part 5. One-Hot Encoding of Nominal Features

In this step, I applied one-hot encoding to the nominal features in the df_final Dataframe, converting them into dummy variables to make the data suitable for machine learning models. First, I identified the nominal features, which were originally stored as strings, and used the get_dummies() function to create binary variables for each category in these features. To avoid multicollinearity, I set the drop_first=True parameter, which removed the first column from each set of dummy variables. This ensures that the encoded variables are not redundant. After creating the dummy variables, I dropped the original nominal columns from df_final and added the newly created dummy variables back into the DataFrame. This process transformed the categorical data into a numerical format, making it ready for analysis and machine learning, while ensuring efficient and accurate encoding.

In [45]:
# Selecting nominal features:
nominal_str = df_final.select_dtypes(include=['object']).columns

# Creating dummy variables:
nominal_hot = pd.get_dummies(df_final[nominal_str], dtype=int, drop_first=True)

# Dropping the original nominal columns from df_final
df_final = df_final.drop(columns=nominal_str)

# Joining the dummy variables back to df_final
df_final = df_final.join(nominal_hot)

---
---

### Part 6. Data standardization and training
In this part, I prepared the dataset for machine learning by splitting it into training and testing sets and standardizing the feature values. First, I separated the target variable y from the feature matrix X. Using train_test_split from the sklearn.model_selection library, I split the data into 75% training data and 25% testing data, ensuring the target variable y was stratified to maintain the same class proportions in both sets.

After splitting, I standardized the feature values using StandardScaler. This scaling technique ensures that all features have a mean of 0 and a standard deviation of 1, which is essential for many machine learning algorithms to perform optimally. The scaler was fitted on the training data and then applied to both the training and test sets to avoid data leakage. This process ensures that the model is trained on normalized data, improving its efficiency and performance during training and testing.

In [46]:
y = df_final['y'].astype(int).values
X = df_final.drop(columns=['y']).values

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Splitting the data into 75% train and 25% test datasets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8, stratify=y)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

---
---

### Part 7. Logistic Regression and Random Forest Classifiers and Accuracies


In this part of the project, I trained two machine learning models—a Logistic Regression classifier and a Random Forest classifier—on standardized data to evaluate their performance. First, I trained a Logistic Regression model using the training data, with random_state set to 10 for reproducibility. After fitting the model, I calculated the accuracy for both the training and test datasets, which provided insight into how well the model generalizes to unseen data.

Next, I trained a Random Forest classifier, also with random_state set to 10, to compare its performance with the Logistic Regression model. I similarly calculated and printed the training and test accuracies to evaluate the model's fit and ability to predict on new data.

This comparison between two different models showcases my ability to implement, train, and evaluate multiple machine learning algorithms, leveraging accuracy metrics to assess their performance and draw insights on model effectiveness.

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Logistic Regression model:
lr = LogisticRegression(random_state=10)

# Training the model on the training data
lr.fit(X_train, y_train)

# Computing accuracy:
train_lr = accuracy_score(y_train, lr.predict(X_train))
test_lr = accuracy_score(y_test, lr.predict(X_test))

print(f'Logistic regression training accuracy: {train_lr:.3f}')
print(f'Logistic regression test accuracy: {test_lr:.3f}')

#Random forest model:
rf = RandomForestClassifier(random_state=10)

# Training the model on the training data
rf.fit(X_train, y_train)

# Computing accuracy:
train_rf = accuracy_score(y_train, rf.predict(X_train))
test_rf = accuracy_score(y_test, rf.predict(X_test))

print(f'Random forest training accuracy: {train_rf:.3f}')
print(f'Random forest test accuracy: {test_rf:.3f}')

Logistic regression training accuracy: 0.660
Logistic regression test accuracy: 0.660
Random forest training accuracy: 0.977
Random forest test accuracy: 0.888


#### Part 7 markdown: 
1. Training and test accuracy comparison:
- **Logistic regression** : Training accuracy(0.660) and test accuracy(0.660) of logistic regression model are same. This means that model is not overfitted it performs same on both training and test data.
- **Random forest** : Training accuracy(0.977) is higher than test accuracy(0.888). This means that model is slightly overfitted, it performs well on training data but its performance drops on unseen data. But since the test accuracy rate is 88.8%% it shows sufficient generalization. 
  
2. Classifier performance comparison:
Although logistic regression shows consistency throughout training and test data, random forest model delivers higher test accuracy rate(88.8%). This shows that random forest will capture more pattern, leading to better forecasting rather than logistic regression.

3. Since random forest significantly outperforms logistic regression, this suggests the presence of nonlinearities in the dataset. The lower performance of logistic regression can likely be attributed to its assumption of linearity, whereas random forest's ability to model nonlinearity resulted in better performance. This implies that the relationships between the features and the dependent variable in this dataset are not linear, and random forest appears to be preffered choise to modeling these complex patterns.

---
---